In [1]:
#!/usr/bin/env sage -python

from sage.all import *
import sys
import math
import random

In [2]:
# p = 3
# q = 2^5 = 32
N = 11
d = 3   # this does not accomplish q > (6d + 1)*p, but who cares

In [3]:
Zx.<x> = ZZ[]

In [4]:
def mult(f, g):
    return (f * g) % (x^N - 1)

In [5]:
def modp(f, p):
    Zp.<z> = PolynomialRing(Integers(p))
    f1 = Zx(Zp(f))
    return f1

In [6]:
def inverse_mod_prime(f, p):
    Zp.<z> = PolynomialRing(Integers(p))
    ZPphi.<Z> = Zp.quotient(z^N - 1)
    fp = Zx(lift(1 / ZPphi(f)))
    return fp

In [7]:
def test_1():
    # Wiki test
    f = -1 + x + x^2 -x^4 + x^6+ x^9- x^10
    p = 5
    fp = inverse_mod_prime(f, p)
    print(fp)
    print(mult(fp, f))
    Zp.<z> = PolynomialRing(Integers(p))
    print(Zp(mult(fp, f)))

test_1()

2*x^10 + 4*x^8 + x^7 + 4*x^6 + 4*x^5 + 2*x^4 + 2*x^3 + 4*x^2 + 4*x + 4
5*x^9 + 5*x^8 + 5*x^7 + 5*x^6 - 5*x^5 + 10*x^3 + 5*x^2 + 1
1


In [8]:
def inverse_mod_power2(f, q):
    t = 2
    f1 = inverse_mod_prime(f, 2)
    while True:
        if t >= q:
            Zq.<z> = PolynomialRing(Integers(q))
            f1 = Zx(Zq(f1))
            return f1
        else:
            t = t**2
            f1 = mult(f1, (2 - mult(f, f1)))
            f1 = modp(f1, t)

In [9]:
def test_2():
    # Wiki test
    f = -1 + x + x^2 -x^4 + x^6+ x^9- x^10
    q = 32
    fq = inverse_mod_power2(f, q)
    print(fq)
    print(mult(fq, f))
    Zq.<z> = PolynomialRing(Integers(q))
    print(Zq(mult(fq, f)))
    
    
test_2()

30*x^10 + 18*x^9 + 20*x^8 + 22*x^7 + 16*x^6 + 15*x^5 + 4*x^4 + 16*x^3 + 6*x^2 + 9*x + 5
32*x^8 + 32*x^5 + 32*x^4 + 32*x + 33
1


In [10]:
def balancedmod(f,p):
    coef = []
    for i in range(N):
        new_coef = ((f[i] + p//2) % p) - p//2
        if new_coef == -p//2:
            new_coef = p//2
        coef.append(new_coef)
    
    return Zx(coef)

In [11]:
def test_3():
    f = 1 + 31*x + 32*x^2 + 33*x^3 - x^4
    p = 32
    f1 = balancedmod(f,p)
    print(f1)
    
test_3()

-x^4 + x^3 - x + 1


In [12]:
#---------------------------------------------------------------------#

In [24]:
def random_gen(d1, d2):
    # d1 is number of 1s, d2 is number of -1s among the coefficients
    
    lst = list(range(N))
    nonzero = random.sample(lst, d1 + d2)
    
    f = 0
    for i in range(d1):
        f += x^(nonzero[i])
    for i in range(d2):
        f -= x^(nonzero[d1 + i])
        
    return f

In [26]:
def test_5():
    print(random_gen(d1=2, d2=3))
    
test_5()

x^8 + x^7 - x^6 - x^5 - x^3


In [31]:
def keys(p, q):
    
    f = random_gen(d + 1, d)
    g = random_gen(d, d)
    
    # f = -1 + x + x^2 -x^4 + x^6+ x^9- x^10
    # g = -1 + x^2 + x^3 + x^5 -x^8 -x^10
    
    try:
        fp = inverse_mod_prime(f, p)
        fq = inverse_mod_power2(f, q)
        
        h = mult((p * fq), g)
        h = modp(h, q)
        h = balancedmod(h,q)
        
        public_key = h
        private_key = (f, fp)
        
        return public_key, private_key
        
    except ZeroDivisionError:
        print("inverse doesn\'t exist, try another f")
        return None


In [56]:
def test_6():
    # Wiki test
    # using hardcoded f, g in keys(p, q)
    p, q = 3, 32
    pub, priv = keys(p, q)
    print("h =", pub)
    print("f =", priv[0])
    print("fp =", priv[1])
    print(priv)

test_6()

h = -15*x^10 - 8*x^9 + 10*x^8 + 9*x^7 + 11*x^6 + 13*x^5 + 12*x^4 + 14*x^3 + 7*x^2 - 11*x - 10
f = -x^10 + x^8 - x^7 + x^5 + x^3 - x^2 + 1
fp = 2*x^10 + 2*x^9 + 2*x^8 + x^6 + 2*x^5 + 2*x^4 + 2*x^3 + 2*x^2 + 2*x + 2
(-x^10 + x^8 - x^7 + x^5 + x^3 - x^2 + 1, 2*x^10 + 2*x^9 + 2*x^8 + x^6 + 2*x^5 + 2*x^4 + 2*x^3 + 2*x^2 + 2*x + 2)


In [57]:
def encryption(m, public_key, p, q):
    m = balancedmod(m,p)
    
    r = random_gen(d, d)
    # r = -1 + x^2 + x^3 + x^4 - x^5 - x^7
    
    h = public_key
    
    e = mult(r, h) + m
    e = modp(e, q)
    
    return e

In [63]:
def test_7():
    # Wiki test
    # using hardcoded r in encryption(m, public_key, p, q)
    m = -1 + x^3 -x^4 -x^8 + x^9 +x^10
    pub = 16*x^10 - 13*x^9 + 12*x^8 - 13*x^7 + 15*x^6 - 8*x^5 + 12*x^4 - 12*x^3 - 10*x^2 - 7*x + 8
    p, q = 3, 32
    print(encryption(m, pub, p, q))

test_7()

6*x^10 + x^9 + 21*x^8 + 5*x^7 + 28*x^6 + 25*x^5 + 28*x^4 + 28*x^3 + 10*x^2 + 4*x + 4


In [64]:
def decryption(e, private_key, p, q):
    f, fp = private_key[0], private_key[1]
    
    a = balancedmod(mult(f, e), q)
    print(a)
    b = balancedmod(a, p)
    print(b)
    c = balancedmod(mult(b, fp), p)
    return c
    

In [66]:
def test_8():
    e = 19*x^10 + 6*x^9 + 25*x^8 + 7*x^7 + 30*x^6 + 16*x^5 + 14*x^4 + 24*x^3 + 26*x^2 + 11*x + 14
    f = -x^10 + x^9 + x^6 - x^4 + x^2 + x - 1
    fp = 2*x^9 + x^8 + 2*x^7 + x^5 + 2*x^4 + 2*x^3 + 2*x + 1
    private_key = [f, fp]
    p, q = 3, 32
    print("m =", decryption(e, private_key, p, q))

test_8()

-7*x^10 - 3*x^9 + 5*x^8 + 7*x^7 + 6*x^6 + 7*x^5 + 10*x^4 - 11*x^3 - 10*x^2 - 7*x + 3
-x^10 - x^8 + x^7 + x^5 + x^4 + x^3 - x^2 - x
m = x^10 + x^9 - x^8 - x^4 + x^3 - 1


In [84]:
def test_global():
    p, q = 3, 32
    m = random_gen(4, 3)
    print("m =", m)
    
    while True:
        try:
            pub, priv = keys(p, q)
            break
        except TypeError:
            continue
    
    e = encryption(m, pub, p, q)
    print("e =", e)
    c = decryption(e, priv, p, q)
    print("c =", c)
    print("m == c is", m==c)

test_global()

m = x^10 + x^9 - x^8 - x^5 + x^4 - x^2 + x
e = 6*x^10 + 12*x^8 + 8*x^7 + 20*x^6 + 10*x^5 + 22*x^4 + 18*x^3 + 19*x^2 + 17*x + 29
3*x^10 + 5*x^9 + 3*x^8 + 6*x^7 - 2*x^6 + 7*x^5 - 4*x^4 - 9*x^3 - x^2 - 7*x
-x^9 + x^6 + x^5 - x^4 - x^2 - x
c = x^10 + x^9 - x^8 - x^5 + x^4 - x^2 + x
m == c is True
